In [1]:
#!pip install scipy
#!pip install sklearn
#!pip install h5py

In [2]:
import torch
from scipy.stats import norm
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from sklearn.mixture import GaussianMixture as GMM
import numpy as np
import os
!pip install torchsummary
import  torchsummary as tt
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import scipy.io
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


import numpy as np
import h5py
import math
import time
from random import shuffle
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import pickle
import numpy as np
import os
!pip install mat73
import mat73
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import tqdm
from tqdm.notebook import tqdm

In [ ]:
# CL-Network
class CustomNeuralNetworkS15(nn.Module):
    def __init__(self):
        super().__init__()
        super(CustomNeuralNetworkS15, self).__init__()
        Stacked_Layers=1
        self.lstm1 = nn.LSTM(int(W/2), 100, Stacked_Layers, batch_first=True)
        self.lstm2 = nn.LSTM(int(W/2), 100, Stacked_Layers, batch_first=True)
        self.lstm3 = nn.LSTM(int(W/2), 100, Stacked_Layers, batch_first=True)
        self.lstm4 = nn.LSTM(int(W/2), 100, Stacked_Layers, batch_first=True)
        self.Flatten = nn.Flatten()
        self.FC1=nn.Linear(400,100)
        self.FC2=nn.Linear(100,1)
        self.LR1 = nn.LeakyReLU(0.2)
        self.LR2 = nn.LeakyReLU(0.2)
        self.LR3 = nn.ReLU()
        self.Sig=nn.Sigmoid()

    def forward(self, x):

        out1,state = self.lstm1(x[:,:,0:int(W/2)])
        out2,state = self.lstm2(x[:,:,int(W/2):2*int(W/2)])
        out3,state = self.lstm3(x[:,:,2*int(W/2):3*int(W/2)])
        out4,state = self.lstm4(x[:,:,3*int(W/2):4*int(W/2)])
        LL=torch.cat((out1,out2,out3,out4),dim=2)
        LLA=self.LR1(LL)
        outCL=self.Flatten(LLA)
        outNF=self.FC1(outCL)
        LLB=self.LR1(outNF)        
        outF=self.FC2(LLB)
        outF=self.LR3(outF)
        return outF


In [ ]:
# Three Dataset exists, 'A', 'B', 'C'
W=2000
S=4
for RM_N in tqdm(['B'],desc='for RM datasets A, B, and C'):
    cwd='C:/Users/15404/Documents/Conda Codes/Rotor/'
    # Rotor data
    Rot_Data=np.load(cwd+RM_N+'_RM_S4_'+str(W)+'.npy')
    # Samples in each Rotor data case
    Class_N=np.cumsum(np.load(cwd+RM_N+'_RM_Classes_'+str(W)+'.npy'))
    # Minimum is
    print('Min RM class samples:\t',np.min(np.load(cwd+RM_N+'_RM_Classes_'+str(W)+'.npy')))
    # we thus take 
    Class_L=int(np.floor(np.min(np.load(cwd+RM_N+'_RM_Classes_'+str(W)+'.npy'))/100))*100
    print('RM data case length is thu staken as:\t',Class_L)
    N=torch.tensor(Rot_Data[0:Class_L,:],device='cuda').reshape((Class_L,1,int(S*W/2))).float()
    Rot_Data.shape
    
    # Training
    #Many Class_N
    #ModelB=0
    #ModelC=2
    for tt in tqdm(range(3,4),desc='RM '+RM_N+' class-by-class training'):
        #print('RM '+RM_N+' Class '+str(tt+1))
        DD1=Class_L-100
        #int(np.floor((Class_L-100)/2))
        N=torch.tensor(Rot_Data[0:Class_L-100,:],device='cuda').reshape((Class_L-100,1,int(S*W/2))).float()
        D=torch.tensor(Rot_Data[int(Class_N[tt]):Class_N[tt]+DD1,:],device='cuda').reshape((DD1,1,int(S*W/2))).float()
        #D1=torch.tensor(Rot_Data[int(Class_N[tt]):Class_N[tt]+DD1,:],device='cuda').reshape((DD1,1,int(S*W/2))).float()
        ## vAL
        NV=torch.tensor(Rot_Data[Class_L-100:Class_L,:],device='cuda').reshape((100,1,int(S*W/2))).float()
        DV=torch.tensor(Rot_Data[Class_N[tt]+DD1:Class_N[tt]+DD1+100,:],device='cuda').reshape((100,1,int(S*W/2))).float()
        
        GG=80
        GG1=0
        Y=torch.tensor(np.zeros((Class_L-100,1)), device='cuda').float()
        Y[:,0]=GG1
        Y1=torch.tensor(np.zeros((Class_L-100,1)), device='cuda').float()
        Y1[:,0]=GG
    
    
        YV=torch.tensor(np.zeros((100,1)), device='cuda').float()
        YV[:,0]=GG1
        YV1=torch.tensor(np.zeros((100,1)), device='cuda').float()
        YV1[:,0]=GG
    
        X=torch.cat((N,D),dim=0)
        #,D1),dim=0)
        XV=torch.cat((NV,DV),dim=0)
        y=torch.cat((Y,Y1),dim=0)
        yv=torch.cat((YV,YV1),dim=0)
        from torch.utils.data import TensorDataset, DataLoader
        dataset = TensorDataset(X, y)
        data_loader = DataLoader(dataset, 200, shuffle=True)
        
    ###########
        Net=CustomNeuralNetworkS15().to(device=device)
        Net.train()
        epoch=1000
        crit=torch.nn.functional.mse_loss
        L=[]
        lam=0
        D_solver = torch.optim.Adam(Net.parameters(), lr=5e-4, betas=(0.9, 0.999))
        pr=1
        zet=0
        L_V=[1000]
        L_T=[]
        pp=0
        for i in range(epoch):
            for XX,yy in data_loader:
                pred=Net.forward(XX)
                loss=crit(pred,yy) 
                D_solver.zero_grad()
                loss.backward()
                D_solver.step()
                if zet==0:
                    L_T.append(loss.item())
                if loss.item()>max(L_T):
                    Net=CustomNeuralNetworkS15().to(device=device)
                    Net.train()  
                    D_solver = torch.optim.Adam(Net.parameters(), lr=5e-4, betas=(0.9, 0.999))
                    zet=0
                    L_T=[]
                L_T.append(loss.item())
            predA=Net.forward(XV)
            lossA=crit(predA,yv) 

            
            if lossA.item()<=np.min(L_V):
                torch.save(Net,cwd+'Models/NT/Model'+RM_N+str(tt+1)+'_W_'+str(W)+'.pth')
                pp=pp+1
            L_V.append(lossA.item())
            zet+=1
        print(loss.item())

for RM datasets A, B, and C:   0%|          | 0/1 [00:00<?, ?it/s]

Min RM class samples:	 768
RM data case length is thu staken as:	 700


RM B class-by-class training:   0%|          | 0/1 [00:00<?, ?it/s]

2.2730139335180866e-06


In [ ]:
cwd+'Models/NT/Model'+RM_N+str(tt+1)+'_W_'+str(W)+'.pth'

'C:/Users/15404/Documents/Conda Codes/Rotor/Models/NT/ModelB4_W_2000.pth'